### Config

In [152]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from itertools import cycle

## Yearly data

In [153]:
timegpt = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/TimeGPT/Year/M3_yearly_simple_forecasts.parquet')

In [154]:
chronos = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/Chronos/Chronos Year/M3_Chronos_year_forecast.parquet')
chronos.drop(columns = ['Low', 'High'], inplace = True)
chronos.rename(columns = {'Series_ID' : 'Series', 'Median' : 'Chronos'}, inplace = True)

In [155]:
moirai = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/MOIRAI/MOIRAI Year/M3_moirai_year_forecasts.parquet')
moirai['Series'] = moirai['Series'].astype(int)
moirai['Date2'] = chronos['Date']

moirai.drop(columns = ['Date'], inplace = True)

moirai.rename(columns = {'Date2' : 'Date','Series_ID' : 'Series', 'Median_Forecast' : 'Moirai'}, inplace = True)

#### Create common dataframe for eaiser analysis

In [156]:
yearly = pd.merge(timegpt, chronos, on = ['Series', 'Date'], how='left')
yearly = pd.merge(yearly, moirai, on = ['Series', 'Date'], how='left')

## Quarterly Data

In [157]:
timegptQ = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/TimeGPT/Quarter/M3_quarter_simple_forecasts.parquet')
timegptQ.reset_index(inplace = True)
timegptQ.drop(columns = ['index'], inplace = True)

In [158]:
chronosQ = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/Chronos/Chronos Quarter/M3_Chronos_quarter_forecast.parquet')
chronosQ['Date2'] = timegptQ['Date']

chronosQ.drop(columns = ['Low', 'High', 'Date'], inplace = True)
chronosQ.rename(columns = {'Date2' : 'Date','Series_ID' : 'Series', 'Median' : 'Chronos'}, inplace = True)

In [159]:
moiraiQ = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/MOIRAI/MOIRAI Quarter/M3_moirai_quarter_forecasts.parquet')
moiraiQ['Series'] = moiraiQ['Series'].astype(int)

moiraiQ['Date2'] = timegptQ['Date']
moiraiQ.drop(columns = ['Date'], inplace = True)
moiraiQ.rename(columns = {'Date2' : 'Date','Series_ID' : 'Series', 'Median_Forecast' : 'Moirai'}, inplace = True)

### Create common dataframe

In [160]:
quarterly = pd.merge(timegptQ, chronosQ, on = ['Series', 'Date'], how='left')
quarterly = pd.merge(quarterly, moiraiQ, on = ['Series', 'Date'], how='left')

## Monthly Data

In [161]:
timegptM = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/TimeGPT/Month/M3_month_simple_forecasts.parquet')

In [162]:
chronosM = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/Chronos/Chronos Month/M3_Chronos_month_forecast.parquet')
chronosM.drop(columns = ['Low', 'High'], inplace = True)
chronosM.rename(columns = {'Series_ID' : 'Series', 'Median' : 'Chronos'}, inplace = True)

In [163]:
moiraiM = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/MOIRAI/MOIRAI Month/M3_moirai_month_forecasts.parquet')
moiraiM['Series'] = moiraiM['Series'].astype(int)
moiraiM.rename(columns = {'Median_Forecast' : 'Moirai'}, inplace = True)

### Create common dataframe

In [164]:
monthly = pd.merge(timegptM, chronosM, on = ['Series', 'Date'], how='left')
monthly = pd.merge(monthly, moiraiM, on = ['Series', 'Date'], how='left')

## Merge all

In [165]:
concatenated_df = pd.concat([yearly, quarterly, monthly])
# Sort the concatenated dataframe by series and then date
sorted_df = concatenated_df.sort_values(['Series', 'Date'])

## Add additional information for easier eval
Add Naïve 2, the deseasonalized random walk, and actual values

### Yearly

In [166]:
def read_all_sheets(excel_path):
    """
    Reads all sheets from an Excel file and combines them into a single DataFrame.
    
    Args:
    - excel_path (str): The path to the Excel file.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the combined data from all sheets.
    """
    # Use pandas to read the Excel file and get sheet names
    xls = pd.ExcelFile(excel_path)
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    # Iterate over all sheet names
    for sheet_name in xls.sheet_names:
        # Read the current sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        
        # Optionally, add a column to indicate the source sheet, if needed
        df['Sheet'] = sheet_name
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list into one
    combined_yearly = pd.concat(dfs, ignore_index=True)
    
    return combined_yearly

excel_path = '/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3Forecast.xls'
combined_yearly = read_all_sheets(excel_path)
# Rename columns
combined_yearly.rename(columns = {0 : 'Series', 1 : 'NF'}, inplace = True)

# Change series to int in the same fashion as the other dataframes
combined_yearly['Series'] = combined_yearly['Series'].str.replace('N', '')
combined_yearly['Series'] = combined_yearly['Series'].str.replace(' ', '')
combined_yearly['Series'] = combined_yearly['Series'].astype(int)
# Only yearly data for this evaluation
combined_yearly = combined_yearly.loc[combined_yearly['Series'] <= 645].copy()
# Drop not necessary columns
cols_to_drop = list(range(8, 19+1))
combined_yearly.drop(columns = cols_to_drop, inplace = True)

In [167]:
# Change position of sheet column
cols = combined_yearly.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_df = combined_yearly[cols]

# Rename columns so that 1 is the first forecast period
combined_yearly.rename(columns=lambda x: x - 1 if isinstance(x, int) else x, inplace=True)

# Melt the dataframe i.e. unpivot
df_long_year = pd.melt(combined_yearly, id_vars=['Sheet', 'NF' , 'Series'], value_name='value', var_name='ForecastPeriod')   

# Sort the dataframe
df_long_year = df_long_year.sort_values(by=['Sheet', 'Series', 'ForecastPeriod']).copy()

### Quarterly

In [168]:
def read_all_sheets(excel_path):
    """
    Reads all sheets from an Excel file and combines them into a single DataFrame.
    
    Args:
    - excel_path (str): The path to the Excel file.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the combined data from all sheets.
    """
    # Use pandas to read the Excel file and get sheet names
    xls = pd.ExcelFile(excel_path)
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    # Iterate over all sheet names
    for sheet_name in xls.sheet_names:
        # Read the current sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        
        # Optionally, add a column to indicate the source sheet, if needed
        df['Sheet'] = sheet_name
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list into one
    combined_df_quarter = pd.concat(dfs, ignore_index=True)
    
    return combined_df_quarter

excel_path = '/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3Forecast.xls'
combined_df_quarter = read_all_sheets(excel_path)
# Rename columns
combined_df_quarter.rename(columns = {0 : 'Series', 1 : 'NF'}, inplace = True)

# Change series to int in the same fashion as the other dataframes
combined_df_quarter['Series'] = combined_df_quarter['Series'].str.replace('N', '')
combined_df_quarter['Series'] = combined_df_quarter['Series'].str.replace(' ', '')
combined_df_quarter['Series'] = combined_df_quarter['Series'].astype(int)
# Only yearly data for this evaluation
combined_df_quarter = combined_df_quarter.loc[(combined_df_quarter['Series'] >= 646) & (combined_df_quarter['Series'] <= 1401)].copy()
# Drop not necessary columns
combined_df_quarter
cols_to_drop = list(range(10, 19+1))
combined_df_quarter.drop(columns = cols_to_drop, inplace = True)

In [169]:
# Change position of sheet column
cols = combined_df_quarter.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_df_quarter = combined_df_quarter[cols]

# Rename columns so that 1 is the first forecast period
combined_df_quarter.rename(columns=lambda x: x - 1 if isinstance(x, int) else x, inplace=True)

# Melt the dataframe i.e. unpivot
df_long_quarter = pd.melt(combined_df_quarter, id_vars=['Sheet', 'NF' , 'Series'], value_name='value', var_name='ForecastPeriod')   

# Sort the dataframe
df_long_quarter = df_long_quarter.sort_values(by=['Sheet', 'Series', 'ForecastPeriod']).copy()

### Monthly

In [170]:
def read_all_sheets(excel_path):
    """
    Reads all sheets from an Excel file and combines them into a single DataFrame.
    
    Args:
    - excel_path (str): The path to the Excel file.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the combined data from all sheets.
    """
    # Use pandas to read the Excel file and get sheet names
    xls = pd.ExcelFile(excel_path)
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    # Iterate over all sheet names
    for sheet_name in xls.sheet_names:
        # Read the current sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        
        # Optionally, add a column to indicate the source sheet, if needed
        df['Sheet'] = sheet_name
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list into one
    combined_df_month = pd.concat(dfs, ignore_index=True)
    
    return combined_df_month

excel_path = '/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3Forecast.xls'
combined_df_month = read_all_sheets(excel_path)
# Rename columns
combined_df_month.rename(columns = {0 : 'Series', 1 : 'NF'}, inplace = True)

# Change series to int in the same fashion as the other dataframes
combined_df_month['Series'] = combined_df_month['Series'].str.replace('N', '')
combined_df_month['Series'] = combined_df_month['Series'].str.replace(' ', '')
combined_df_month['Series'] = combined_df_month['Series'].astype(int)

combined_df_month = combined_df_month.loc[(combined_df_month['Series'] >= 1402) & (combined_df_month['Series'] <= 2829)].copy()

In [171]:
# Change position of sheet column
cols = combined_df_month.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_df_month = combined_df_month[cols]

# Rename columns so that 1 is the first forecast period
combined_df_month.rename(columns=lambda x: x - 1 if isinstance(x, int) else x, inplace=True)

# Melt the dataframe i.e. unpivot
df_long_month = pd.melt(combined_df_month, id_vars=['Sheet', 'NF' , 'Series'], value_name='value', var_name='ForecastPeriod')   

# Sort the dataframe
df_long_month = df_long_month.sort_values(by=['Sheet', 'Series', 'ForecastPeriod']).copy()

### Change format to add as columns

In [172]:
# Use pivot_table to transform the DataFrame
pivot_df_year = df_long_year.pivot_table(index=['Series', 'ForecastPeriod'],  # Adjust or expand index list based on needs
                          columns='Sheet',                     # Values in 'Sheet' become new column headers
                          values='value',                      # Fill new columns with values from 'value'
                          aggfunc='sum')                       # Aggregate function in case of duplicates

# Reset the index if you want 'Series' and 'ForecastPeriod' back as columns (optional)
pivot_df_year.reset_index(inplace=True)

In [173]:
# Use pivot_table to transform the DataFrame
pivot_df_quarter = df_long_quarter.pivot_table(index=['Series', 'ForecastPeriod'],  # Adjust or expand index list based on needs
                          columns='Sheet',                     # Values in 'Sheet' become new column headers
                          values='value',                      # Fill new columns with values from 'value'
                          aggfunc='sum')                       # Aggregate function in case of duplicates

# Reset the index if you want 'Series' and 'ForecastPeriod' back as columns (optional)
pivot_df_quarter.reset_index(inplace=True)

In [174]:
# Use pivot_table to transform the DataFrame
pivot_df_month = df_long_month.pivot_table(index=['Series', 'ForecastPeriod'],  # Adjust or expand index list based on needs
                          columns='Sheet',                     # Values in 'Sheet' become new column headers
                          values='value',                      # Fill new columns with values from 'value'
                          aggfunc='sum')                       # Aggregate function in case of duplicates

# Reset the index if you want 'Series' and 'ForecastPeriod' back as columns (optional)
pivot_df_month.reset_index(inplace=True)

In [175]:
pivot_df_year['Date'] = yearly['Date']
pivot_df_quarter['Date'] = quarterly['Date']
pivot_df_month['Date'] = monthly['Date']

In [176]:
previous_methods = pd.concat([pivot_df_year, pivot_df_quarter, pivot_df_month])
previous_methods.drop(columns = ['AAM1', 'AAM2'], inplace = True) # These columns not used for yearly, exclude for simplicity

## Final Merge for Complete Data

In [177]:
complete_df = pd.merge(sorted_df, previous_methods, on = ['Series', 'Date'], how='left')

In [151]:
complete_df.to_parquet('complete_forecast_data.parquet', engine='pyarrow', compression='snappy', index=False)